<center>
    <h1>NBA Playoff Predictions</h1>
    <h3>Neel Shah</h3>
</center>

# Introduction

# Data Collection

# Data Processing

# Exploratory Data Analysis & Visualization

# Modeling: Analysis, Hypothesis Testing, & Machine Learning

# Interpretation: Insight & Policy Decision